In [1]:
item = {'kit_type': 'Timing Chain',
 'kit_no': 1,
 'kit_type_code': '5724',
 'kit_manufacturer': 'MELLING',
 'kit_part_number': '715F',
 'kit_url': 'https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724&Lennon=1',
 'kit_picture': '',
 'kit_part_code': '8306448',
 'kit_join_mpntcpc': 'MELLING;715F;5724;8306448',
 'kit_not_found': 'no',
 'kit_choose': '',
 'kit_json_sidenote': '{}',
 'kit_info_url': 'https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724',
 'kit_json_footnote': '{"0": "Category: Timing Chain"}',
 'kit_oem': '3L8Z6268AA;L30512201',
 'kit_json_src': '{"0": "https://www.rockauto.com/info/583/715F-1-ANG.jpg"}',
 'kit_json_price': '{"default": "Out of Stock"}'}

item

{'kit_type': 'Timing Chain',
 'kit_no': 1,
 'kit_type_code': '5724',
 'kit_manufacturer': 'MELLING',
 'kit_part_number': '715F',
 'kit_url': 'https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724&Lennon=1',
 'kit_picture': '',
 'kit_part_code': '8306448',
 'kit_join_mpntcpc': 'MELLING;715F;5724;8306448',
 'kit_not_found': 'no',
 'kit_choose': '',
 'kit_json_sidenote': '{}',
 'kit_info_url': 'https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724',
 'kit_json_footnote': '{"0": "Category: Timing Chain"}',
 'kit_oem': '3L8Z6268AA;L30512201',
 'kit_json_src': '{"0": "https://www.rockauto.com/info/583/715F-1-ANG.jpg"}',
 'kit_json_price': '{"default": "Out of Stock"}'}

In [2]:
import requests

import sys
sys.path.append('../00.Tools')
from configuration import generate_header, generate_proxy

In [3]:
response = requests.get(item['kit_info_url'],
                        headers=generate_header(),
                        proxies=generate_proxy(),
                        timeout=(10, 15),
                        allow_redirects=False)

response.status_code

200

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(response.text, 'html5lib')

with open('./9.kit_specification.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html class="no-js REPLACEHTMLCLASS" lang="en" onclick="">
 <head>
  <meta content="all" name="robots"/>
  <meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
  <link href="/jslib/combined/css/desktopmoreinfo.css?1749384923" rel="stylesheet" type="text/css"/>
  <script>
   if(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};
_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift();for

In [6]:
from parsel import Selector

In [7]:
selector = Selector(text=str(soup))

selector

<Selector query=None data='<html class="no-js REPLACEHTMLCLASS" ...'>

In [8]:
list_text = selector.xpath('//span[@class="la-btn-v"]/text()').get().strip()

list_text

'Continue Shopping'

In [9]:
list_text not in ['Continue Shopping', 'Add to Cart']

False

In [10]:
from lxml import etree
import re
import html

In [11]:
dict_key, list_tr = {}, selector.xpath('//table[@class="moreinfotable"]/tbody/tr')
for tr in list_tr:
    list_td = tr.xpath('./td')
    if not list_td:
        continue
    list_text = [etree.tostring(td.root).decode('utf-8') for td in list_td]
    list_text = [re.sub(r'<.*?>', '', text) for text in list_text]
    list_text = [re.sub(r' +', ' ', text) for text in list_text]
    list_text = [html.unescape(text).strip() for text in list_text]

    key = list_text[0]

    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_specification, dict_count = {}, {}
for tr in list_tr:
    list_td = tr.xpath('./td')
    if not list_td:
        continue
    list_text = [etree.tostring(td.root).decode('utf-8') for td in list_td]
    list_text = [re.sub(r'<.*?>', '', text) for text in list_text]
    list_text = [re.sub(r' +', ' ', text) for text in list_text]
    list_text = [html.unescape(text).strip() for text in list_text]

    key, value = list_text[0], '' if len(list_text) == 1 else '\n'.join(list_text[1:])

    if dict_key[key] == 1:
        dict_specification[key] = 'nan' if value == '' else value
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_specification[f'{key}_{dict_count[key]}'] = 'nan' if value == '' else value

dict_specification

{'Chain Length_1': '21.75 IN',
 'Chain Length_2': '552.0 mm',
 'Chain Material': 'Hardened Steel',
 'Chain Pitch_1': '0.25 in',
 'Chain Pitch_2': '6.0 MM',
 'Chain Type': 'Silent Chain Flexible',
 'Chain Width_1': '0.43 IN',
 'Chain Width_2': '11.0 MM',
 'Link Quantity': '174',
 'Roller Quantity': '1',
 'Row Quantity': '1',
 'Timing Mark Quantity': '0',
 'Timing Marks': 'No'}

In [12]:
import json

In [13]:
{'kit_type': item['kit_type'],
 'kit_no': item['kit_no'],
 'kit_type_code': item['kit_type_code'],
 'kit_manufacturer': item['kit_manufacturer'],
 'kit_part_number': item['kit_part_number'],
 'kit_url': item['kit_url'],
 'kit_picture': item['kit_picture'],
 'kit_part_code': item['kit_part_code'],
 'kit_join_mpntcpc': item['kit_join_mpntcpc'],
 'kit_not_found': item['kit_not_found'],
 'kit_choose': item['kit_choose'],
 'kit_json_sidenote': item['kit_json_sidenote'],
 'kit_info_url': item['kit_info_url'],
 'kit_json_footnote': item['kit_json_footnote'],
 'kit_oem': item['kit_oem'],
 'kit_json_src': item['kit_json_src'],
 'kit_json_price': item['kit_json_price'],
 'kit_json_specification': json.dumps(dict_specification)}

{'kit_type': 'Timing Chain',
 'kit_no': 1,
 'kit_type_code': '5724',
 'kit_manufacturer': 'MELLING',
 'kit_part_number': '715F',
 'kit_url': 'https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724&Lennon=1',
 'kit_picture': '',
 'kit_part_code': '8306448',
 'kit_join_mpntcpc': 'MELLING;715F;5724;8306448',
 'kit_not_found': 'no',
 'kit_choose': '',
 'kit_json_sidenote': '{}',
 'kit_info_url': 'https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724',
 'kit_json_footnote': '{"0": "Category: Timing Chain"}',
 'kit_oem': '3L8Z6268AA;L30512201',
 'kit_json_src': '{"0": "https://www.rockauto.com/info/583/715F-1-ANG.jpg"}',
 'kit_json_price': '{"default": "Out of Stock"}',
 'kit_json_specification': '{"Chain Length_1": "21.75 IN", "Chain Length_2": "552.0 mm", "Chain Material": "Hardened Steel", "Chain Pitch_1": "0.25 in", "Chain Pitch_2": "6.0 MM", "Chain Type": "Silent Chain Flexible", "Chain Width_1": "0.43 IN", "Chain Width_2": "11.0 MM", "Link Quantity": "174", "Roller